In [1]:
from glob import glob
import numpy as np
from matplotlib import pylab as plt
import cv2
import time
import os
import torchvision.transforms as transforms
import torch
import tensorflow as tf
print(tf.__version__)
import tensorflow.keras.layers as layers
from tensorflow.keras.models import Model
import pandas as pd

2.6.2


In [2]:
df = pd.read_csv('../input/videopaths/datapaths.csv',header=0,index_col='Unnamed: 0')

In [3]:
df.replace('audio-files-for-miraclvc1/ph1-13','audio-files-for-miraclvc1/ph1-1',inplace=True)
for i in list(set(df['spectrograms'])):
    if not os.path.exists('../input/'+i+'.jpg'):
        df.replace(i,'audio-files-for-miraclvc1/ph1-1',inplace=True)

In [4]:
class Aud2LipsDataset(torch.utils.data.Dataset):
    def __init__(self):
        self.imgs = df
    def __getitem__(self, idx):
        # load lipimage,spectrogram and target:lipmovement image
        lip_path = self.imgs['lippath'][idx]
        spect_path='../input/'+self.imgs['spectrograms'][idx]+'.jpg'
        lipmove_path=self.imgs['videopath'][idx]
        #resize and load as tensor
        lipimg=cv2.cvtColor(cv2.imread(lip_path), cv2.COLOR_BGR2RGB)
        spectimg=cv2.cvtColor(cv2.imread(spect_path),cv2.COLOR_BGR2RGB)
        lipmoveimg=cv2.cvtColor(cv2.imread(lipmove_path), cv2.COLOR_BGR2RGB)
        lipimg = lipimg[250:400,150:400]
        lipimg = cv2.resize(lipimg,(170,150))
        spectT = cv2.resize(spectimg,(80, 150))
        lipmoveimg=lipmoveimg[250:400,150:400]
        
        imageA = np.concatenate((lipimg,spectT),axis=1)
        imageB = lipmoveimg
        return imageA,imageB
    
    def __len__(self):
        return len(self.imgs)
dataset = Aud2LipsDataset()

In [5]:
def load_batch(batch_size):
    #path1=sorted(glob('../input/building/facade/train_picture/*'))
    #path2=sorted(glob('../input/building/facade/train_label/*'))
    n_batches=int(len(df)/batch_size)
  
    for i in range(n_batches):
        #batch1=path1[i*batch_size:(i+1)*batch_size]
        #batch2=path2[i*batch_size:(i+1)*batch_size]
        img_A,img_B=[],[]
        for idx in range(i*batch_size,(i+1)*batch_size):
            img1,img2 = dataset[idx]
            img_A.append(img1)#picture
            img_B.append(img2)#label
      
        img_A=np.array(img_A)/255
        img_B=np.array(img_B)/255
    
        yield img_A,img_B #return generator

In [6]:
class pix2pix():
    def __init__(self):
        self.img_rows= 150
        self.img_cols= 250
        self.channels=3
        self.img_shape=(self.img_rows,self.img_cols,self.channels)
    
        patch=int(self.img_rows/(2**4)) # 16
        self.disc_patch=(10,16,1)
    
        self.gf=64
        self.df=64
    
        optimizer=tf.keras.optimizers.Adam(0.0002,0.5)
    
        self.discriminator=self.build_discriminator()
        #self.discriminator.summary()
        self.discriminator.compile(loss='binary_crossentropy',
                              optimizer=optimizer)
    
        self.generator=self.build_generator()
        #self.generator.summary()
    
        img_A=layers.Input(shape=self.img_shape)#picture--label
        img_B=layers.Input(shape=self.img_shape)#label--real
    
        img=self.generator(img_A)
    
        self.discriminator.trainable=False
    
        valid=self.discriminator([img,img_A])
    
        self.combined=Model(img_A,valid)
        self.combined.compile(loss='binary_crossentropy',
                              optimizer=optimizer)
    
    def build_generator(self):
        def conv2d(layer_input,filters,f_size=(4,4),bn=True):
            d=layers.Conv2D(filters,kernel_size=f_size,strides=(2,2),padding='same')(layer_input)
            d=layers.LeakyReLU(0.2)(d)
            if bn:
                d=layers.BatchNormalization()(d)
            return d
    
        def deconv2d(layer_input,skip_input,filters,f_size=(4,4),dropout_rate=0):
            u=layers.UpSampling2D((2,2))(layer_input)
            u=layers.Conv2D(filters,kernel_size=f_size,strides=(1,1),padding='same',activation='relu')(u)
            if dropout_rate:
                u=layers.Dropout(dropout_rate)(u)
            u=layers.BatchNormalization()(u)
            u=layers.Concatenate()([u,skip_input])
            return u
    
        d0=layers.Input(shape=self.img_shape)
    
        d1=conv2d(d0,self.gf,bn=False) 
        d1 = tf.keras.layers.Resizing(256,320)(d1)
        d2=conv2d(d1,self.gf*2)
        d2 = tf.keras.layers.Resizing(128,160)(d2)
        d3=conv2d(d2,self.gf*4)  
        d3 = tf.keras.layers.Resizing(64,80)(d3)
        d4=conv2d(d3,self.gf*8) 
        d4 = tf.keras.layers.Resizing(32,40)(d4)
        d5=conv2d(d4,self.gf*8)
        d5 = tf.keras.layers.Resizing(16,20)(d5)
        d6=conv2d(d5,self.gf*8)        
    
        d7=conv2d(d6,self.gf*8)         
    
        u1=deconv2d(d7,d6,self.gf*8,dropout_rate=0.5)
        u2=deconv2d(u1,d5,self.gf*8,dropout_rate=0.5)   
        u3=deconv2d(u2,d4,self.gf*8,dropout_rate=0.5)   
        u4=deconv2d(u3,d3,self.gf*4)   
        u5=deconv2d(u4,d2,self.gf*2)   
        u6=deconv2d(u5,d1,self.gf)     
        u7=layers.UpSampling2D((2,2))(u6)
    
        output_img=layers.Conv2D(self.channels,kernel_size=(4,4),strides=(1,1),padding='same',activation='tanh')(u7)
        output_img = tf.keras.layers.Resizing(150,250)(output_img)
        
        return Model(d0,output_img)
  
    def build_discriminator(self):
        def d_layer(layer_input,filters,f_size=(4,4),bn=True):
            d=layers.Conv2D(filters,kernel_size=f_size,strides=(2,2),padding='same')(layer_input)
            d=layers.LeakyReLU(0.2)(d)
            if bn:
                d=layers.BatchNormalization()(d)
            return d
    
        img_A=layers.Input(shape=self.img_shape)
        img_B=layers.Input(shape=self.img_shape)
    
        combined_imgs=layers.Concatenate(axis=-1)([img_A,img_B])
    
        d1=d_layer(combined_imgs,self.df,bn=False)
        d2=d_layer(d1,self.df*2)
        d3=d_layer(d2,self.df*4)
        d4=d_layer(d3,self.df*8)
    
        validity=layers.Conv2D(1,kernel_size=(4,4),strides=(1,1),padding='same',activation='sigmoid')(d4)
    
        return Model([img_A,img_B],validity)
  
    def train(self,epochs,batch_size):
        valid=np.ones((batch_size,)+self.disc_patch)
        fake=np.zeros((batch_size,)+self.disc_patch)
    
        for epoch in range(epochs):
            start=time.time()
            for batch_i,(img_A,img_B) in enumerate(load_batch(batch_size)):
                gen_imgs=self.generator.predict(img_A)
        
                d_loss_real = self.discriminator.train_on_batch([img_B, img_A], valid)
                d_loss_fake = self.discriminator.train_on_batch([gen_imgs, img_A], fake)
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        
                g_loss = self.combined.train_on_batch(img_A,valid)

                if batch_i % 50 == 0:
                    print ("[Epoch %d] [Batch %d] [D loss: %f] [G loss: %f]" % (epoch,
                                                                                batch_i,
                                                                                d_loss,
                                                                                g_loss))
                if batch_i % 1000 == 0:
                    tf.keras.models.save_model(self.generator, 'trained-gan-model',
                                               overwrite=True, include_optimizer=True,
                                               save_format=None)
            #self.sample_images(epoch)
            print('Time for epoch {} is {} sec'.format(epoch,time.time()-start))
      
    def sample_images(self, epoch):
        r, c = 1, 1
        img_A, img_B = next(iter(load_batch(1)))
        fake_A = self.generator.predict(img_A)

        gen_imgs = np.concatenate([img_A, fake_A, img_B])

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        titles = ['Condition','Generated', 'Original']
        fig, axs = plt.subplots(r, c)
        cnt = 0
        plt.imshow(gen_imgs[cnt])
        plt.set_title(titles[i])
        plt.axis('off')
        
        fig.savefig("./%d.png" % (epoch))
        plt.show()

In [7]:
if __name__ == '__main__':
    gan = pix2pix()
    #gan.generator = tf.keras.models.load_model('../input/gan-aud2lips/trained-gan-model/', compile=True)
    tf.keras.models.save_model(gan.generator, 'trained-gan-model', overwrite=True, include_optimizer=True, save_format=None)
    gan.train(epochs=8, batch_size=4)

2022-02-09 13:11:05.196627: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-09 13:11:05.197613: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-09 13:11:05.198276: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-09 13:11:05.199018: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

ead from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-09 13:11:09.326853: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14959 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


2022-02-09 13:11:12.927081: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


2022-02-09 13:11:17.761123: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


2022-02-09 13:11:18.862435: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


...

[Epoch 5] [Batch 3750] [D loss: 0.000000] [G loss: 0.142240]


[Epoch 5] [Batch 3800] [D loss: 0.000000] [G loss: 0.146524]


[Epoch 5] [Batch 3850] [D loss: 0.000000] [G loss: 0.162105]


[Epoch 5] [Batch 3900] [D loss: 0.000000] [G loss: 0.133268]


[Epoch 5] [Batch 3950] [D loss: 0.000000] [G loss: 0.146536]


#### Loading Model

In [ ]:
#gan.generator = tf.keras.models.load_model('../input/gan-aud2lips/trained-gan-model/', compile=True)

In [ ]:
plt.imshow(dataset[4][1])

In [ ]:
cf = dataset[4][0][None,]/255
gan.generator.predict(cf)